In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
import os
from tqdm.notebook import tqdm

In [3]:
from metrics import multiclass_stats

In [4]:
os.chdir('yelp_academic')

In [5]:
br = pd.read_csv('data/business_ambience.csv')

In [6]:
br.head()

,photo_id,touristy,hipster,romantic,divey,intimate,upscale
0,ZlTwL6uWx6rW_L9Df5RT8A,False,False,True,False,True,False
1,fHbSMxueQfXFRb9e-6bJuw,False,False,False,True,False,False
2,74oWvVVIjms9LjfHQOgxMQ,False,False,False,False,False,True
3,QY6c1OKsIpujF4MDHQdbag,False,True,False,False,False,False
4,0AYEzNJYFF2PeXo71cpKuw,False,True,False,False,False,False


In [7]:
feature_cols = ['touristy', 'hipster', 'romantic', 'divey', 'intimate', 'upscale']

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train_files, X_test_files, y_train, y_test = train_test_split(br.photo_id, br[feature_cols], train_size = 0.9, random_state=420, stratify=br[feature_cols])

In [10]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [11]:
with open('img_objects.pickle', 'rb') as io:
    objects = pickle.load(io)

In [12]:
trf_features = np.load('data/transfer_features/densenet_features.npz', allow_pickle = True)['arr_0'][()]

In [13]:
list(objects.items())[:2]

[(('LYiu06twTYN5_HndA_b-Cg',),
  [([(166.24751, 158.46727), (223.51633, 339.36807)], 46, 0.9968928),
   ([(115.754234, 100.15254), (175.79247, 289.48053)], 44, 0.9934689),
   ([(5.7081475, 143.40843), (57.751545, 327.2468)], 46, 0.99128866),
   ([(84.24994, 52.882652), (170.16812, 136.35014)], 1, 0.9865284),
   ([(4.960636, 227.64551), (220.60417, 397.3526)], 67, 0.96494514),
   ([(0.8416633, 4.649881), (220.56383, 127.64469)], 79, 0.78191566),
   ([(0.0, 222.43925), (18.95716, 284.204)], 47, 0.67385864),
   ([(100.82414, 291.31363), (144.11972, 337.40964)], 61, 0.5718027)]),
 (('EyRUH511mIh3We4Ce0PFVg',),
  [([(268.95145, 183.03993), (300.0, 228.78711)], 8, 0.79613465)])]

In [14]:
vector_size = len(COCO_INSTANCE_CATEGORY_NAMES)

In [15]:
binary_feature_vectors = {}

In [16]:
for name, boxes in tqdm(objects.items()):
    confidence_vector = np.zeros(vector_size)
    counts_vector = np.zeros(vector_size)
    for box in boxes:
        if box:
            _, idx, confidence = box
            confidence_vector[idx] = max(confidence_vector[idx], confidence)
            counts_vector[idx] += 1
    binary_feature_vectors[name[0]] = np.concatenate((confidence_vector, counts_vector))

  0%|          | 0/84222 [00:00<?, ?it/s]

In [17]:
all_vectors = np.array(list(binary_feature_vectors.values()))

In [18]:
empty_columns = []

trans_arr = all_vectors.T
for i in range(trans_arr.shape[0]):
    if np.all(trans_arr[i] == trans_arr[i][0]):
        empty_columns.append(i)

In [19]:
for c in empty_columns[::-1]:
    if c < len(COCO_INSTANCE_CATEGORY_NAMES):
        del COCO_INSTANCE_CATEGORY_NAMES[c]
    all_vectors = np.delete(all_vectors, c, 1)

In [20]:
names = list(binary_feature_vectors.keys())

In [21]:
features = {
    names[i]: np.concatenate((trf_features[names[i]], all_vectors[i])) for i in range(len(names))
}

In [22]:
X_train, X_test = [], []

In [23]:
for filename in tqdm(X_train_files):
    X_train.append(features[filename])

  0%|          | 0/29635 [00:00<?, ?it/s]

In [24]:
for filename in tqdm(X_test_files):
    X_test.append(features[filename])

  0%|          | 0/3293 [00:00<?, ?it/s]

In [25]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [26]:
X_test.shape

(3293, 1184)

In [27]:
y_train = y_train.to_numpy(dtype='int')
y_test = y_test.to_numpy(dtype='int')

In [28]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import multilabel_confusion_matrix, classification_report, hamming_loss, f1_score, roc_auc_score

In [29]:
lr_clf = make_pipeline(StandardScaler(),
                       MultiOutputClassifier(
                           LogisticRegression(max_iter=10000,
                                              random_state=42,
                                              class_weight='balanced'))
                      ).fit(X_train, y_train)

In [30]:
y_pred = lr_clf.predict(X_test)

In [31]:
report, stats = multiclass_stats(y_test, y_pred)
stats

/home/ophelion/venv/yelp/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'hamming_score': 0.3581890879643689,
 'hamming_loss': 0.3062556938961433,
 'f1_score': 0.4249066619590178,
 'roc_auc_score': 0.6742230438060256}

In [32]:
print(report)

              precision    recall  f1-score   support

           0       0.39      0.64      0.48       730
           1       0.55      0.65      0.59      1253
           2       0.23      0.59      0.33       464
           3       0.23      0.67      0.34       304
           4       0.22      0.61      0.32       440
           5       0.36      0.71      0.48       585

   micro avg       0.34      0.65      0.45      3776
   macro avg       0.33      0.65      0.42      3776
weighted avg       0.39      0.65      0.47      3776
 samples avg       0.37      0.65      0.45      3776



In [33]:
multilabel_confusion_matrix(y_test, y_pred)

array([[[1838,  725],
        [ 266,  464]],

       [[1373,  667],
        [ 441,  812]],

       [[1897,  932],
        [ 188,  276]],

       [[2299,  690],
        [  99,  205]],

       [[1873,  980],
        [ 171,  269]],

       [[1988,  720],
        [ 172,  413]]])